In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Load and unzip 
# !cp "/content/drive/My Drive/Face_Mask_Detection/FMD_DATASET_Split.zip" "Face_Mask"
# !unzip Face_Mask -d data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#importing dependencies related to nn
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#importing dependencies related to image transformations
import torchvision
from torchvision import transforms
from PIL import Image

#importing dependencies related to data loading
from torchvision import datasets
from torch.utils.data import DataLoader

#importing Tensorboard for data visualization
from torch.utils.tensorboard import SummaryWriter

from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sns   

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#defining image transformations

image_transforms = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
        transforms.Resize(size=(224,224)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ]),
    # Validation does not use augmentation
    'test':
    transforms.Compose([
        transforms.Resize(size=(224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ])
}

In [ ]:
# loading data
data = {
    'train':
    datasets.ImageFolder(root='/content/data/FMD_DATASET/Train', transform=image_transforms['train']),
    'test':
    datasets.ImageFolder(root='/content/data/FMD_DATASET/Test', transform=image_transforms['test']),
}

# Dataloader iterators, used for making batches
dataloaders = {
    'train': DataLoader(data['train'], batch_size=100, shuffle=True),
    'test': DataLoader(data['test'], batch_size=100, shuffle=True)
}

In [ ]:
#loading MobileNetv2
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth
100%|██████████| 230M/230M [00:03<00:00, 68.8MB/s]


In [ ]:
#freezing the initial layers of MobileNetv2
for param in model.parameters():
    param.requires_grad = False

In [ ]:
#adding our own classifier
model.fc = nn.Sequential(
                      nn.Linear(2048, 3),
                      nn.LogSoftmax(dim=1))

In [ ]:
# !cp "/content/drive/My Drive/Face_Mask_Detection/Weights/Res-FaceMDN-epc-33-acc-0.9626373626373627.pth" "Trained_weight.pth"
# model = torch.load('Trained_weight.pth', map_location=lambda storage, loc: storage)
#model.load_state_dict(pretrained)

In [ ]:
#checking trainable parameters
# print(model)
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

6147


In [ ]:
#checking GPU Avaibility
if torch.cuda.is_available():
    print('training on GPU')
    device = torch.device("cuda:0")
else:
    print('training on CPU')
    device = torch.device("cpu")

training on GPU


In [ ]:
#training data
from tqdm.notebook import tqdm #for loading bars

def train(model, optimizer, loss_fn, train_loader, val_loader, epochs,device):
    writer = SummaryWriter("saves/logs")
    for epoch in tqdm(range(epochs)):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        num_correct_train = 0 
        num_examples_train = 0
        for batch in (train_loader):
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            #print(targets)
            output = model(inputs)
            #print(output)
            loss = loss_fn(output, targets)
            # loss.requires_grad = True
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
            correct_train = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets).view(-1)
            #y_pred.extend(correct.cpu().numpy())
            num_correct_train += torch.sum(correct_train).item()
            num_examples_train += correct_train.shape[0]
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        y_true = []
        y_pred = []
        for batch in (val_loader):
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            _, predicted = torch.max(output.data, 1)
            # print(predicted)
            targets = targets.to(device)
            y_true.extend(targets.cpu().numpy())
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)          
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets).view(-1)
            y_pred.extend(predicted.cpu().numpy())
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)
        cf_matrix = confusion_matrix(y_true, y_pred)
        #print(cf_matrix)
        class_names = ('incorrect_mask', 'with_mask', 'without_mask')
        # Create pandas dataframe
        dataframe = pd.DataFrame(cf_matrix, index=class_names, columns=class_names)
        plt.figure(figsize=(8, 6))
      
        # Create heatmap
        sns.heatmap(dataframe, annot=True, cbar=None,cmap="YlGnBu",fmt="d")

        plt.title("Confusion Matrix"), plt.tight_layout()

        plt.ylabel("True Class"), 
        plt.xlabel("Predicted Class")
        plt.show()
        print('Epoch: {}, Training Loss: {:.4f}, Train Accuracy = {:.4f}, Validation Loss: {:.4f}, Val Accuracy = {:.4f}'.format(epoch+34, training_loss, num_correct_train / num_examples_train, valid_loss, num_correct / num_examples))
        
        #save model
        if(training_loss<0.1):
            torch.save(model,"/content/drive/My Drive/Face_Mask_Detection/Weights/Res-FaceMDN-epc-"+str(epoch+34)+"-acc-"+str(num_correct / num_examples)+".pth")
        
        #tensorBoard save log
        writer.add_scalar('Loss', loss.item(), epoch)
        writer.add_scalar('Train/Loss', training_loss, epoch)
        writer.add_scalar('Test/Loss', valid_loss, epoch)
        writer.add_scalar('Test/Accuracy',num_correct / num_examples, epoch)
        writer.flush()

In [ ]:
#testing data
def test_model(model):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloaders['test']:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('correct: {:d}  total: {:d}'.format(correct, total))
    print('accuracy = {:f}'.format(correct / total))

In [ ]:
#put model to GPU (if available)
model=model.to(device)

In [ ]:
#choosing optimizer and loss Function
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss = torch.nn.CrossEntropyLoss()

In [ ]:
#train the model first 100 epochs
train(model, optimizer,loss,dataloaders['train'],dataloaders['test'],100,device)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#test the model
test_model(model)

In [ ]:
#save the model
torch.save(model, "./firstHundread.pth")